# TDS Week 2 Assignment

# Imports

In [99]:
import os
import re
import time
import json
import random
import ast
from datetime import datetime

import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

import wikipedia as wk

from urllib.parse import urlencode, urljoin
import urllib.request

import tabula
from tabula import convert_into

from geopy.geocoders import Nominatim

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Following are wrong api
# from urllib.parse import urldevelop
# from urllib.parse import urlformulate

## Load tables in excel

In [4]:
# url = https://www.timeanddate.com/weather/india/chennai/ext
    
# there are 2 tables on the webpage
# Open Excel
# Data -> Get Data -> From other sources -> from web ->  paste url
# -> connect -> OK -> table view -> select table -> Load 
# -> then queries connection side tab will open
# -> double click Table 0 from side tab -> modify columns
# -> in Home tab select close & load
# -> Right click on table and select refresh

## BBC Weather

In [12]:
# This code snippet takes city name as input and it hits the BBC 
# Weahter API with a request for location ID. This location ID is 
# used as input in the next part of the code to scrape weather 
# forecast for the city using this location ID.

# Web scraping might not be legal always. It is a good idea to 
# check the terms of the website you plan to scrape before proceeding. 
# Also, if your code requests a url from a server multiple times, 
# it is a good practice to either cache your requests, or insert a 
# timed delay between consecutive requests.

In [13]:
url = "https://www.bbc.com/weather/5380748"
#  it uses api to retrieve data, we use that url

In [14]:
test_city = "New York"
location_url = 'https://locator-service.api.bbci.co.uk/locations?' + urlencode({
   'api_key': 'AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv',
   's': test_city,
   'stack': 'aws',
   'locale': 'en',
   'filter': 'international',
   'place-types': 'settlement,airport,district',
   'order': 'importance',
   'a': 'true',
   'format': 'json'
})

location_url

# 'https://locator-service.api.bbci.co.uk/locations?api_key=AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv&amp;s=New+York&amp;stack=aws&amp;locale=en&amp;filter=international&amp;place-types=settlement%2Cairport%2Cdistrict&amp;order=importance&amp;a=true&amp;format=json'

'https://locator-service.api.bbci.co.uk/locations?api_key=AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv&s=New+York&stack=aws&locale=en&filter=international&place-types=settlement%2Cairport%2Cdistrict&order=importance&a=true&format=json'

In [15]:
result = requests.get(location_url).json()
result

# {'response': {'results': {'results': [{'id': '5128581',
#      'name': 'New York',
#      'container': 'United States',
#      'containerId': 6252001,
#      'language': 'en',
#      'timezone': 'America/New_York',
#      'country': 'US',
#      'latitude': 40.71427,
#      'longitude': -74.00597,
#      'placeType': 'settlement'},
#     {'id': '2641508',
#      'name': 'New York',
#      'container': 'Lincolnshire',
#      'containerId': 2644486,
#      'language': 'en',
#      'timezone': 'Europe/London',
#      'country': 'GB',
#      'latitude': 53.07897,
#      'longitude': -0.14008,
#      'placeType': 'settlement',
#      'topicId': 'cr0mejz7n26t'}],
#    'totalResults': 2}}}

{'response': {'results': {'results': [{'id': '5128581',
     'name': 'New York',
     'container': 'United States',
     'containerId': 6252001,
     'language': 'en',
     'timezone': 'America/New_York',
     'country': 'US',
     'latitude': 40.71427,
     'longitude': -74.00597,
     'placeType': 'settlement'},
    {'id': '2641508',
     'name': 'New York',
     'container': 'Lincolnshire',
     'containerId': 2644486,
     'language': 'en',
     'timezone': 'Europe/London',
     'country': 'GB',
     'latitude': 53.07897,
     'longitude': -0.14008,
     'placeType': 'settlement',
     'topicId': 'cr0mejz7n26t'}],
   'totalResults': 2}}}

In [16]:
result['response']['results']['results'][0]['id']

'5128581'

In [17]:
def getlocid(city):
    city = city.lower() # convert city name to lowercase to standardize format
    # Convert into an API call using URL encoding
    location_url = 'https://locator-service.api.bbci.co.uk/locations?' + urlencode({
      'api_key': 'AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv',
      's': city,
      'stack': 'aws',
      'locale': 'en',
      'filter': 'international',
      'place-types': 'settlement,airport,district',
      'order': 'importance',
      'a': 'true',
      'format': 'json'
    })
    result = requests.get(location_url).json()
    locid = result['response']['results']['results'][0]['id']
    return locid

In [18]:
getlocid('Toronto')

'6167865'

## BBC Scrape with BeautifulSoup

In [19]:
# This example scrapes the BBC weather website for any specific city, 
# and collects weather forecast for the next 14 days and saves it 
# as a csv file.

In [20]:
# 'https://www.bbc.com/weather/1275339' # 1275339 is Mumbai city ID
city_id = 1275339
url      = f'https://www.bbc.com/weather/{city_id}'
response = requests.get(url)
response.ok

True

In [22]:
soup = bs(response.content,'html.parser') 

In [23]:
daily_high_values = soup.findAll('span', attrs={'class': 'wr-day-temperature__high-value'}) # block-type: span; identifier type: class; and class name: wr-day-temperature__high-value 
len(daily_high_values)

13

In [24]:
daily_high_values[0]

<span class="wr-day-temperature__high-value"><span class="wr-value--temperature"><span class="wr-value--temperature--c">31°</span><span class="wr-hide"> </span><span class="wr-value--temperature--f">88°</span></span></span>

In [25]:
daily_low_values  = soup.find_all('span', attrs={'class': 'wr-day-temperature__low-value'})
len(daily_low_values)

14

In [26]:
daily_summary = soup.find('div', attrs={'class': 'wr-day-summary'})
daily_summary.text

'Thundery showers and a gentle breezeLight cloud and a gentle breezeLight cloud and a gentle breezeLight rain showers and a gentle breezeThundery showers and a gentle breezeThundery showers and a moderate breezeThundery showers and a moderate breezeThundery showers and a moderate breezeThundery showers and a moderate breezeThundery showers and a moderate breezeThundery showers and a moderate breezeThundery showers and a moderate breezeThundery showers and a moderate breezeThundery showers and a moderate breeze'

In [28]:
daily_high_values[0].text.strip()

'31° 88°'

In [29]:
daily_high_values[0].text.strip().split()[0]

'31°'

In [30]:
daily_high_values_list = [daily_high_values[i].text.strip().split()[0] for i in range(len(daily_high_values))]
print(daily_high_values_list)

['31°', '31°', '31°', '30°', '30°', '30°', '29°', '29°', '29°', '29°', '29°', '29°', '29°']


In [31]:
daily_low_values_list = [daily_low_values[i].text.strip().split()[0] for i in range(len(daily_low_values))]

In [32]:
daily_summary_list = re.findall('[a-zA-Z][^A-Z]*', daily_summary.text) #split the string on uppercase
print(daily_summary_list)

['Thundery showers and a gentle breeze', 'Light cloud and a gentle breeze', 'Light cloud and a gentle breeze', 'Light rain showers and a gentle breeze', 'Thundery showers and a gentle breeze', 'Thundery showers and a moderate breeze', 'Thundery showers and a moderate breeze', 'Thundery showers and a moderate breeze', 'Thundery showers and a moderate breeze', 'Thundery showers and a moderate breeze', 'Thundery showers and a moderate breeze', 'Thundery showers and a moderate breeze', 'Thundery showers and a moderate breeze', 'Thundery showers and a moderate breeze']


In [36]:
datelist = pd.date_range(datetime.today(), periods=len(daily_high_values)).tolist()
datelist

[Timestamp('2022-06-23 18:49:00.364335', freq='D'),
 Timestamp('2022-06-24 18:49:00.364335', freq='D'),
 Timestamp('2022-06-25 18:49:00.364335', freq='D'),
 Timestamp('2022-06-26 18:49:00.364335', freq='D'),
 Timestamp('2022-06-27 18:49:00.364335', freq='D'),
 Timestamp('2022-06-28 18:49:00.364335', freq='D'),
 Timestamp('2022-06-29 18:49:00.364335', freq='D'),
 Timestamp('2022-06-30 18:49:00.364335', freq='D'),
 Timestamp('2022-07-01 18:49:00.364335', freq='D'),
 Timestamp('2022-07-02 18:49:00.364335', freq='D'),
 Timestamp('2022-07-03 18:49:00.364335', freq='D'),
 Timestamp('2022-07-04 18:49:00.364335', freq='D'),
 Timestamp('2022-07-05 18:49:00.364335', freq='D')]

In [37]:
datelist = [datelist[i].date().strftime('%y-%m-%d') for i in range(len(datelist))]
datelist

['22-06-23',
 '22-06-24',
 '22-06-25',
 '22-06-26',
 '22-06-27',
 '22-06-28',
 '22-06-29',
 '22-06-30',
 '22-07-01',
 '22-07-02',
 '22-07-03',
 '22-07-04',
 '22-07-05']

In [38]:
zipped = zip(datelist, daily_high_values_list, daily_low_values_list, daily_summary_list)

In [39]:
df = pd.DataFrame(list(zipped), columns=['Date', 'High','Low', 'Summary'])

In [40]:
display(df)

,Date,High,Low,Summary
0,22-06-23,31°,24°,Thundery showers and a gentle breeze
1,22-06-24,31°,26°,Light cloud and a gentle breeze
2,22-06-25,31°,26°,Light cloud and a gentle breeze
3,22-06-26,30°,26°,Light rain showers and a gentle breeze
4,22-06-27,30°,27°,Thundery showers and a gentle breeze
5,22-06-28,30°,26°,Thundery showers and a moderate breeze
6,22-06-29,29°,26°,Thundery showers and a moderate breeze
7,22-06-30,29°,26°,Thundery showers and a moderate breeze
8,22-07-01,29°,26°,Thundery showers and a moderate breeze
9,22-07-02,29°,26°,Thundery showers and a moderate breeze


In [41]:
# remove the 'degree' character
df.High = df.High.replace('\°','',regex=True).astype(float)
df.Low  = df.Low.replace('\°','',regex=True).astype(float)

In [42]:
display(df)

,Date,High,Low,Summary
0,22-06-23,31.0,24.0,Thundery showers and a gentle breeze
1,22-06-24,31.0,26.0,Light cloud and a gentle breeze
2,22-06-25,31.0,26.0,Light cloud and a gentle breeze
3,22-06-26,30.0,26.0,Light rain showers and a gentle breeze
4,22-06-27,30.0,27.0,Thundery showers and a gentle breeze
5,22-06-28,30.0,26.0,Thundery showers and a moderate breeze
6,22-06-29,29.0,26.0,Thundery showers and a moderate breeze
7,22-06-30,29.0,26.0,Thundery showers and a moderate breeze
8,22-07-01,29.0,26.0,Thundery showers and a moderate breeze
9,22-07-02,29.0,26.0,Thundery showers and a moderate breeze


In [43]:
#location = soup.find('div', attrs={'class':'wr-c-location'})
location = soup.find('h1', attrs={'id':'wr-location-name-id'})
location.text.split()

['Mumbai', '-', 'Weather', 'warnings', 'issued']

In [44]:
# create a recording
filename_csv = location.text.split()[0]+'.csv'
df.to_csv(filename_csv, index=None)

In [45]:
# filename_xlsx = location.text.split()[0]+'.xlsx'
# df.to_excel(filename_xlsx)

## Scrape IMDB

In [46]:
url = "https://www.imdb.com/chart/top/"
response = requests.get(url)
soup = bs(response.content,'html.parser') 

In [48]:
movie_title = []
movie_year = []
movie_rating = []

In [49]:
imdb_table = soup.find(class_="chart full-width")
movie_titlecolumn = imdb_table.find_all(class_="titleColumn")
movie_ratingscolumn = imdb_table.find_all(class_="ratingColumn imdbRating")

In [50]:
for row in movie_titlecolumn:
    title = row.a.text # tag content extraction
    movie_title.append(title)
    
print(len(movie_title))
print(movie_title[:5])

250
['The Shawshank Redemption', 'The Godfather', 'The Dark Knight', 'The Godfather Part II', '12 Angry Men']


In [51]:
for row in movie_titlecolumn:
    year = row.span.text # tag content extraction
    movie_year.append(year)
    
print(len(movie_year))
print(movie_year[:5])

250
['(1994)', '(1972)', '(2008)', '(1974)', '(1957)']


In [52]:
for row in movie_ratingscolumn:
    rating = row.strong.text # tag content extraction
    movie_rating.append(rating)
    
print(len(movie_rating))
print(movie_rating[:5])

250
['9.2', '9.2', '9.0', '9.0', '8.9']


In [53]:
movie_df = pd.DataFrame({'Movie Title': movie_title, 'Year of Release': movie_year, 'IMDb Rating': movie_rating})
movie_df.head(3)

,Movie Title,Year of Release,IMDb Rating
0,The Shawshank Redemption,(1994),9.2
1,The Godfather,(1972),9.2
2,The Dark Knight,(2008),9.0


In [54]:
movie_df.shape

(250, 3)

## Wikipedia

In [65]:
x = wk.search("Kingsport")
print(len(x), "\n", x)

10 
 ['Kingsport, Tennessee', 'Kingsport (disambiguation)', 'Kingsport Mets', 'Lovecraft Country', 'Kingsport Axmen', 'Tri-Cities, Tennessee', 'Appalachian League', 'Kingsport, Nova Scotia', 'Kingsport City Schools', 'Kingsport–Bristol–Bristol, Tennessee-Virginia Metropolitan Statistical Area']


In [3]:
# ?urlencode
# ?urljoin

### IITM

In [66]:
print(wk.search("IIT Madras")) 

['IIT Madras', 'Indian Institutes of Technology', 'Joint Entrance Examination – Advanced', 'Inter IIT Sports Meet', 'IIT Tirupati', 'List of IIT Madras people', 'IIT Palakkad', 'History of Indian Institutes of Technology', 'IIT Kharagpur', 'Chennai']


In [67]:
print(wk.search("IIT Madras", results=2))

['IIT Madras', 'Indian Institutes of Technology']


In [68]:
print(wk.summary("IIT Madras"))

Indian Institute of Technology Madras (IIT Madras) is a public technical university located in Chennai, Tamil Nadu, India. As one of the Indian Institutes of Technology (IITs), it is recognized as an Institute of National Importance and has been consistently rated as one of India's most prestigious universities. Founded in 1959 with technical and financial assistance from the former government of West Germany, it was the third IIT established by the Government of India. IIT Madras is ranked the top engineering institute in India by the Ministry of Education's National Institutional Ranking Framework since its inception in 2016.IIT Madras is a residential institute that occupies a 2.5-square-kilometre (0.97 sq mi) campus that was formerly part of the adjoining Guindy National Park. The institute has nearly 600 faculty, 10,000 students and 1,250 administrative and supporting staff. Growing ever since it obtained its charter from the Indian Parliament in 1961, much of the campus is a prot

In [69]:
print(wk.summary("IIT Madras", sentences=2))

Indian Institute of Technology Madras (IIT Madras) is a public technical university located in Chennai, Tamil Nadu, India. As one of the Indian Institutes of Technology (IITs), it is recognized as an Institute of National Importance and has been consistently rated as one of India's most prestigious universities.


In [70]:
full_page = wk.page("IIT Madras")

In [71]:
print(full_page.content)

Indian Institute of Technology Madras (IIT Madras) is a public technical university located in Chennai, Tamil Nadu, India. As one of the Indian Institutes of Technology (IITs), it is recognized as an Institute of National Importance and has been consistently rated as one of India's most prestigious universities. Founded in 1959 with technical and financial assistance from the former government of West Germany, it was the third IIT established by the Government of India. IIT Madras is ranked the top engineering institute in India by the Ministry of Education's National Institutional Ranking Framework since its inception in 2016.IIT Madras is a residential institute that occupies a 2.5-square-kilometre (0.97 sq mi) campus that was formerly part of the adjoining Guindy National Park. The institute has nearly 600 faculty, 10,000 students and 1,250 administrative and supporting staff. Growing ever since it obtained its charter from the Indian Parliament in 1961, much of the campus is a prot

In [72]:
print(full_page.url)

https://en.wikipedia.org/wiki/IIT_Madras


In [73]:
print(full_page.references)

['http://www.businessweek.com/smallbiz/news/date/9811/e981130.htm', 'http://www.hindu.com/2009/02/28/stories/2009022854140500.htm', 'http://timesofindia.indiatimes.com/tech/tech-news/IIT-Madras-is-Indias-Stanford/articleshow/46629160.cms', 'http://www.shanghairanking.com/rankings/arwu/2021', 'http://www.thehindu.com/news/cities/chennai/a-helping-hand-to-entrepreneurship/article17856871.ece', 'http://www.thehindu.com/sci-tech/technology/iitm-computer-system-gets-fastest-tag/article3972727.ece', 'http://www.thehindubusinessline.com/news/education/esummit-at-iit-madras-starts-on-april-7/article9620678.ece', 'http://archiv.tu-chemnitz.de/pub/2006/0020/data/MAthesis_EvelynRichter.pdf', 'http://www.oeb.harvard.edu/faculty/mahadevan/mahadevan-oeb.html', 'http://www.csail.mit.edu/user/723', 'http://nms.lcs.mit.edu/~hari/', 'http://ccw.iitm.ac.in/?q=IITMHostels/hinfo', 'http://www.ecell.iitm.ac.in/e-summit/', 'http://nptel.iitm.ac.in/', 'http://nptel.iitm.ac.in/pdf/NPTEL%20Document.pdf', 'http:

In [74]:
print(full_page.title)

IIT Madras


In [75]:
print(full_page.images)

['https://upload.wikimedia.org/wikipedia/commons/f/f2/Blackbuck_white.jpg', 'https://upload.wikimedia.org/wikipedia/commons/1/1b/Brahmaputra_Hostel.jpg', 'https://upload.wikimedia.org/wikipedia/commons/c/c4/Deer_of_IIT_Madras_masatran_0058_0010.jpeg', 'https://upload.wikimedia.org/wikipedia/commons/e/e4/Godavari_Hostel_Entrance.jpg', 'https://upload.wikimedia.org/wikipedia/commons/0/08/Heritage_Center1.jpg', 'https://upload.wikimedia.org/wikipedia/commons/8/85/Pfeil_links.svg', 'https://upload.wikimedia.org/wikipedia/commons/7/76/Pfeil_oben.svg', 'https://upload.wikimedia.org/wikipedia/commons/c/c5/Pfeil_rechts.svg', 'https://upload.wikimedia.org/wikipedia/commons/c/c3/Pfeil_unten.svg', 'https://upload.wikimedia.org/wikipedia/commons/7/71/Sabarmathi_hostel2.jpg', 'https://upload.wikimedia.org/wikipedia/en/4/4a/Commons-logo.svg', 'https://upload.wikimedia.org/wikipedia/en/f/f2/Edit-clear.svg', 'https://upload.wikimedia.org/wikipedia/en/4/41/Flag_of_India.svg', 'https://upload.wikimedia.

In [76]:
print(full_page.images[0])

https://upload.wikimedia.org/wikipedia/commons/f/f2/Blackbuck_white.jpg


In [79]:
#extract html code of wikipedia page based on any search text
html = wk.page("IIT Madras").html().encode("UTF-8") 

In [82]:
df = pd.read_html(html)[4]  
df

,Letter Grade,Grade Points,in Words
0,S,10,Excellent (top students/high performer)
1,A,9,Very Good
2,B,8,Good
3,C,7,Satisfactory Work
4,D,6,Below Average
5,E,4,Poor (but passed)
6,U,0,Failed
7,W,0,Shortage of attendance (usually below 85%)


## Geocode city, get (lat,long)

In [55]:
# If you get a HTTP Error 403 from Nominatim, use your email 
# ID instead of “myGeocoder” in Nominatim(user_agent="myGeocoder")

In [57]:
locator = Nominatim(user_agent="myGeocoder")

In [58]:
location = locator.geocode("Pune")
location

Location(Pune City, Pune, Maharashtra, India, (18.521428, 73.8544541, 0.0))

In [60]:
type(location)

geopy.location.Location

In [61]:
location.raw

{'place_id': 284820555,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'relation',
 'osm_id': 139836,
 'boundingbox': ['39.697215', '39.82119', '-94.899029', '-94.743339'],
 'lat': '39.7686055',
 'lon': '-94.8466322',
 'display_name': 'St. Joseph, Buchanan County, Missouri, United States',
 'class': 'boundary',
 'type': 'administrative',
 'importance': 0.7229305699659045,
 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons/poi_boundary_administrative.p.20.png'}

In [62]:
location.address, location.altitude

('St. Joseph, Buchanan County, Missouri, United States', 0.0)

In [63]:
location = locator.geocode("St. Joseph")
location

Location(St. Joseph, Buchanan County, Missouri, United States, (39.7686055, -94.8466322, 0.0))

In [64]:
location = locator.geocode("Deerfield Beach")
location

Location(Deerfield Beach, Broward County, Florida, United States, (26.318342, -80.0996306, 0.0))

## Scrape PDFs from a Given URL

### Scrape pdf from www.premierleague.com

#### Create folder

In [95]:
# Save contents from url into folder_location
url = 'https://www.premierleague.com/publications'
folder_location = r'premier_league'
if not os.path.exists(folder_location):
    os.mkdir(folder_location)

#### Download all pdf 

In [97]:
response = requests.get(url)
soup = bs(response.text, "html.parser")     

In [98]:
# Loop through all PDF links in the page
for link in soup.select("a[href$='.pdf']"):
    # Local lile name is the same as PDF file name in the URL (ignoring rest of the path)
    # https://premierleague-static-files.s3.amazonaws.com/premierleague/document/2016/07/02/e1648e96-4eeb-456e-8ce0-d937d2bc7649/2011-12-premier-league-season-review.pdf
    filename = os.path.join(folder_location, link['href'].split('/')[-1])
    with open(filename, 'wb') as f:
        f.write(requests.get(urljoin(url,link['href'])).content)

#### Read table from a PDF 

In [102]:
combined_pdf = folder_location + "/This-is-PL-Interactive-Combined.pdf"
df = tabula.read_pdf(combined_pdf,  pages='18') # read page 18

In [103]:
type(df)

list

In [104]:
len(df)

1

In [106]:
type(df[0])

pandas.core.frame.DataFrame

In [107]:
df[0].head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Pos,Unnamed: 3,Club,W,D,L,GD,Pts,Total payment
0,NaN,NaN,NaN,1.0,NaN,Manchester City,32.0,4.0,2.0,79.0,100.0,£149.4m
1,NaN,NaN,NaN,2.0,NaN,Manchester United,25.0,6.0,7.0,40.0,81.0,£149.8m
2,NaN,NaN,NaN,3.0,NaN,Tottenham Hotspur,23.0,8.0,7.0,38.0,77.0,£144.4m
3,NaN,NaN,NaN,4.0,NaN,Liverpool,21.0,12.0,5.0,46.0,75.0,£145.9m
4,NaN,NaN,NaN,5.0,NaN,Chelsea,21.0,7.0,10.0,24.0,70.0,£141.7m


In [108]:
convert_into(combined_pdf, folder_location +"/table_output.csv", output_format="csv",pages = 18,area=[[275,504,640,900]])


In [110]:
df = pd.read_csv(folder_location+"/table_output.csv", encoding= 'unicode_escape')
df.head()

,Pos,Unnamed: 1,Club,W,D,L,GD,Pts,Total payment
0,1.0,NaN,Manchester City,32.0,4.0,2.0,79.0,100.0,£149.4m
1,2.0,NaN,Manchester United,25.0,6.0,7.0,40.0,81.0,£149.8m
2,3.0,NaN,Tottenham Hotspur,23.0,8.0,7.0,38.0,77.0,£144.4m
3,4.0,NaN,Liverpool,21.0,12.0,5.0,46.0,75.0,£145.9m
4,5.0,NaN,Chelsea,21.0,7.0,10.0,24.0,70.0,£141.7m


### Read pdf

In [84]:
os.listdir()

['.ipynb_checkpoints',
 'actor_network.ipynb',
 'Assignment 5 question 7.txt',
 'BBC_weather_scrape.ipynb',
 'Fiona-1.8.21-cp38-cp38-win_amd64.whl',
 'GDAL-3.4.3-cp38-cp38-win_amd64.whl',
 'Geospatial_analysis2.ipynb',
 'Goat_2022.csv',
 'ImdbTextClassification.ipynb',
 'IMDB_Network_dj.ipynb',
 'master_currency.csv',
 'Mumbai.csv',
 'PDF_Scraping_MBCopy.ipynb',
 'rainfall-dataset.csv',
 'SentimentAnalysis_2.xlsx',
 'shortest_path.ipynb',
 'TDS Week 1 Assignment.ipynb',
 'TDS Week 2 Assignment.ipynb',
 'TDS Week 3 Assignment.ipynb',
 'TDS Week 4A Assignment.ipynb',
 'TDS Week 4B Assignment.ipynb',
 'TDS Week 5 Assignment.ipynb.ipynb',
 'TDS_Week_5_Dataset_1_q789.csv',
 'TDS_Week_5_Dataset_2_q10.csv',
 'week 2 scrape table from web.xlsx',
 'week 2 tables dummy.pdf',
 'week 2 tables dummy.xlsx',
 'wikipedia_library.ipynb',
 '~$week 2 tables dummy.xlsx']

In [85]:
df = tabula.read_pdf('week 2 tables dummy.pdf',
                      pages='all',
                      multiple_tables=True)

In [87]:
df[0]

,Andaman & Nicobar1 I9sl0a1nds,49.2,87.1,29.2,2.3,528.8,517.5,365.1
0,Andaman & Nicobar1 I9sl0a2nds,0.0,159.8,12.2,0.0,446.1,537.1,228.9
1,Andaman & Nicobar1 I9sl0a3nds,12.7,144.0,0.0,1.0,235.1,479.9,728.4
2,Andaman & Nicobar1 I9sl0a4nds,9.4,14.7,0.0,202.4,304.5,495.1,502.0
3,Andaman & Nicobar1 I9sl0a5nds,1.3,0.0,3.3,26.9,279.5,628.7,368.7
4,Andaman & Nicobar1 I9sl0a6nds,36.6,0.0,0.0,0.0,556.1,733.3,247.7
5,Andaman & Nicobar1 I9sl0a7nds,110.7,0.0,113.3,21.6,616.3,305.2,443.9
6,Andaman & Nicobar1 I9sl0a8nds,20.9,85.1,0.0,29.0,562.0,693.6,481.4
7,Andaman & Nicobar1 I9sl1a0nds,26.6,22.7,206.3,89.3,224.5,472.7,264.3
8,Andaman & Nicobar1 I9sl1a1nds,0.0,8.4,0.0,122.5,327.3,649.0,253.0


In [90]:
df[1]

,22-06-2023,31,24,Thundery showers and a gentle breeze
0,22-06-2024,31,26,Light cloud and a gentle breeze
1,22-06-2025,31,26,Light cloud and a gentle breeze
2,22-06-2026,30,26,Light rain showers and a gentle breeze
3,22-06-2027,30,27,Thundery showers and a gentle breeze
4,22-06-2028,30,26,Thundery showers and a moderate breeze
5,22-06-2029,29,26,Thundery showers and a moderate breeze


In [91]:
df[2]

,481.1,332.6,388.5,558.2,33.6
0,753.7,666.2,197.2,359.0,160.5
1,326.7,339.0,181.2,284.4,225.0
2,160.1,820.4,222.2,308.7,40.1
3,330.5,297.0,260.7,25.4,344.7
4,320.5,164.3,267.8,128.9,79.2
5,377.6,200.4,264.4,648.9,245.6
6,699.9,428.8,170.7,208.1,196.9
7,337.4,626.6,208.2,267.3,153.5
8,187.1,464.5,333.8,94.5,247.1


In [101]:
len(df)

3

# END